5.1 Image Compression

In [ ]:
import numpy as np
import scipy
from PIL import Image
import matplotlib.pyplot as plt

def load_image(file_path):
    with Image.open(file_path) as img:
        return np.array(img.convert('L'))

def compute_svd(image_matrix):
    U, s, Vt = np.linalg.svd(image_matrix, full_matrices=False)
    return U, s, Vt

def compress_image(U, s, Vt, k):
    S = np.diag(s[:k])
    return np.dot(U[:, :k], np.dot(S, Vt[:k, :]))

def plot_images(images, titles, frobenius_norms, base_filename):
    rows = len(images)
    plt.figure(figsize=(20, 5 * rows))
    for i, (image, title, fro_norm) in enumerate(zip(images, titles, frobenius_norms)):
        plt.subplot(rows, 1, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f'{title}\nFrobenius norm: {fro_norm:.3g}', fontsize=10, pad=3)
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    
def frobenius_norm(original, approx):
    return np.linalg.norm(original - approx)


image_paths = ['ph121_image1.jpg', 'ph121_image2.jpg', 'ph121_image3.jpg', 'ph121_image4.jpg']

for path in image_paths:  
    A = load_image(path)
    U, s, Vt = compute_svd(A)
    ks = [len(s)//4, len(s)//8, len(s)//16, len(s)//32, len(s)//64, len(s)//128, len(s)//256, len(s)//512, len(s)//1024]
    images = [A]  
    titles = ['Original'] 
    frobenius_norms = [0]  
    
    for k in ks:
        A_k = compress_image(U, s, Vt, k)
        images.append(A_k)
        fro_norm = frobenius_norm(A, A_k)
        frobenius_norms.append(fro_norm)
        titles.append(f'k={k}')
    
    filename = f'{path[:-4]}_compression_results.png'
    plot_images(images, titles, frobenius_norms, filename)

5.2 Entaglement Entropy

In [ ]:
from scipy.sparse import csr_matrix
import scipy
from tqdm import tqdm
def ising_sparse(L,h,J,periodic):
  rows = []
  cols = []
  matrix_elements = []
  for b in range(2**L):
    for j in range(1, L + 1):
      a = b ^ (1 << (j - 1))
      rows.append(a)
      cols.append(b)
      matrix_elements.append(-h)

  for a in range(2**L):
    diagonal_value =0
    for j in range(1, L):
        if a & (1 << j) == (a & (1 << (j - 1))) * 2:
            diagonal_value -= J
        else:
            diagonal_value += J

    if periodic:
        if (a & (1 << (L - 1))) == ((a & 1) * (2 ** (L - 1))):
            diagonal_value -= 1
        else:
            diagonal_value += 1
    if diagonal_value != 0:       
        rows.append(a)
        cols.append(a)
        matrix_elements.append(diagonal_value)
  H_sparse = csr_matrix((matrix_elements, (rows, cols)), shape=(2**L, 2**L), dtype=np.float64)
  return H_sparse

solving at representative points in the ferromagnetic and paramagnetic phases and at critical h

In [ ]:
#ferromagnetic
ground_state = scipy.sparse.linalg.eigsh(ising_sparse(10,0.3,1,False), k=1, which='SA', return_eigenvectors=False)[0]
print(ground_state)

In [ ]:
#paramagnetic
ground_state = scipy.sparse.linalg.eigsh(ising_sparse(10,1.7,1,False), k=1, which='SA', return_eigenvectors=False)[0]
print(ground_state)

In [ ]:
#critical h=1
ground_state = scipy.sparse.linalg.eigsh(ising_sparse(10,1,1,False), k=1, which='SA', return_eigenvectors=False)[0]
print(ground_state)

Calculating the entanglement entropy S(l;L)

In [ ]:
from qutip import Qobj, entropy_vn
import numpy as np



H=ising_sparse(12,0.4,1, False)
def entanglement_entropy(H, L, l):

    eigvals, eigvecs = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
    ground_state_vec = eigvecs
    

    ground_state = Qobj(ground_state_vec, dims=[[2] * L, [1]])
    

    subspace_dims = [2] * L  
    subsystem_A = list(range(l)) 
    rho_A = ground_state.ptrace(subsystem_A)

    rho_A_eigenvalues = rho_A.eigenenergies()
    entropy = -sum(eigenval * np.log(eigenval) for eigenval in rho_A_eigenvalues if eigenval>0)
    return entropy





L = 12  
l = 3   
periodic = False 


S_l_L = entanglement_entropy(H, L, l)
print(f"Entanglement entropy S({l};{L}) = {S_l_L}")

Plotting S(l,L) as a function of l for different values of L=[9,10,11,12,13,14]

For open boundary conditions

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm


h_values = [0.4, 1, 1.7]
L_values = [9, 10, 11, 12, 13, 14]


for h in h_values:
    plt.figure(figsize=(8, 5))

    for L in L_values:
        periodic = True
        H = ising_sparse(L, h, 1, periodic=False) 

        entropies = []
        ls = range(1, L)  

        for l in tqdm(ls, desc=f"Calculating Entropy for L={L}, h={h}"):
            entropy = entanglement_entropy(H, L, l)
            entropies.append(entropy)

        plt.plot(ls, entropies, marker='o', label=f'L={L}')

    plt.xlabel('Subsystem size l')
    plt.ylabel('Entanglement entropy S(l, L)')
    plt.title(f'Entanglement Entropy vs. Subsystem Size for h={h} (Periodic Boundary)')
    plt.legend()
    plt.show()

Plot of S(L/2,L)as a function of L

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm


L_values = range(2, 14)
h_values = [0.4, 1, 1.7]


plt.figure(figsize=(8, 5))


for h in h_values:
    entropy_values = []

    for L in tqdm(L_values, desc=f"Calculating Entropy for h={h}"):
        periodic = False
        H = ising_sparse(L, h, 1, periodic)
        

        l = L // 2  
        entropy = entanglement_entropy(H, L, l)
        entropy_values.append(entropy)


    plt.plot(list(L_values), entropy_values, marker='o', label=f'h={h}')


plt.xlabel('System size L')
plt.ylabel('Entanglement entropy S(L/2, L)')
plt.title('Entanglement Entropy S(L/2, L) vs. System Size L (Open Boundary)')
plt.legend()
plt.show()

Now repeat with closed boundary conditions

Entaglement entropy vs subsystem size

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm


h_values = [0.4, 1, 1.7]
L_values = [9, 10, 11, 12, 13, 14]


for h in h_values:

    plt.figure(figsize=(8, 5))

    for L in L_values:
        periodic = True
        H = ising_sparse(L, h, 1, periodic)  


        entropies = []
        ls = range(1, L)  


        for l in tqdm(ls, desc=f"Calculating Entropy for L={L}, h={h}"):
            entropy = entanglement_entropy(H, L, l)
            entropies.append(entropy)

 
        plt.plot(ls, entropies, marker='o', label=f'L={L}')


    plt.xlabel('Subsystem size l')
    plt.ylabel('Entanglement entropy S(l, L)')
    plt.title(f'Entanglement Entropy vs. Subsystem Size for h={h} (Periodic Boundary)')
    plt.legend()
    plt.show()

Plot of S(L/2, L)

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm


h_values = [0.4, 1, 1.7]  
L_values = range(2, 15)   


for h in h_values:
    entropy_values = []

    for L in tqdm(L_values, desc=f"Calculating Entropy for h={h}"):
        periodic = True
        H = ising_sparse(L, h, 1, periodic)  
        
        l = L // 2  
        entropy = entanglement_entropy(H, L, l)
        entropy_values.append(entropy)


    plt.plot(list(L_values), entropy_values, marker='o', label=f'h={h}')


plt.xlabel('System size L')
plt.ylabel('Entanglement entropy S(L/2, L)')
plt.title('Entanglement Entropy S(L/2, L) vs. System Size L (Closed boundary)')
plt.legend()
plt.show()

Entaglement Entropy is much larger since a closed loop increases the number of paths along which quantum correlations can propagate. Closed systems lack edges, so there are no boundary spins that only interact with one neighbor, which would otherwise reduce the entanglement entropy.

Fit of entaglement entropy with function

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
L=14
H=ising_sparse(L,1,1, False)

def fit_function(l, c, C):

    return (c / 3) * np.log((L / np.pi) * np.sin(np.pi * l / L)) + C

entropies = []
ls = range(1,L)
for l in tqdm(ls, desc="Calculating Entropy"):
    entropy = entanglement_entropy(H, L, l)
    entropies.append(entropy)



ls = np.arange(1, L)  


popt, pcov = curve_fit(fit_function, ls, entropies)


c_fit, C_fit = popt


print(f"c: {c_fit}, C: {C_fit}")


plt.figure(figsize=(8, 5))
plt.plot(ls, entropies, 'o', label='Computed S(l, L)')
plt.plot(ls, fit_function(ls, *popt), '-', label='Fit S(l, L)')
plt.xlabel('Subsystem size l')
plt.ylabel('Entanglement entropy S(l, L)')
plt.title('Fit of Entanglement Entropy vs. Subsystem Size')
plt.legend()
plt.show()

Similar for highest energy of the hamiltonian for open boundary conditions

Function computing the entaglement entropy


In [ ]:
from qutip import Qobj
import scipy.sparse.linalg
import numpy as np
L = 12  
l = 3   
H=ising_sparse(L,1,1,False)
def entanglement_entropy(H, L, l, highest=False):
    which_eig = 'LA' if highest else 'SA'
    eigvals, eigvecs = scipy.sparse.linalg.eigsh(H, k=1, which=which_eig)
    state_vec = eigvecs
    state = Qobj(state_vec, dims=[[2] * L, [1]])
    subspace_dims = [2] * L  
    subsystem_A = list(range(l)) 
    rho_A = state.ptrace(subsystem_A)
    entropy = entropy_vn(rho_A, base=np.e)
    return entropy
S_l_L_highest = entanglement_entropy(H, L, l, highest=True)
print(f"Entanglement entropy for the highest excited state S({l};{L}) = {S_l_L_highest}")

plot of entaglement entropy vs susbsystem size with highest excited state

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
h_values = [0.4, 1, 1.7]
L_values = [9, 10, 11, 12, 13, 14]

for h in h_values:
    plt.figure(figsize=(8, 5))
    for L in L_values:
        periodic = True
        H = ising_sparse(L, h, 1, False) 
        entropies = []
        ls = range(1, L) 
        for l in tqdm(ls, desc=f"Calculating Entropy for L={L}, h={h}"):
            entropy = entanglement_entropy(H, L, l, highest= True)
            entropies.append(entropy)
        plt.plot(ls, entropies, marker='o', label=f'L={L}')
    plt.xlabel('Subsystem size l')
    plt.ylabel('Entanglement entropy S(l, L)')
    plt.title(f'Entanglement Entropy vs. Subsystem Size for h={h} (Periodic Boundary)')
    plt.legend()
    plt.show()

Plot of S(L/2,L)as a function of L with highest energy state and open boundary conditions

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

L_values = range(2, 14)
h_values = [0.4, 1, 1.7]
plt.figure(figsize=(8, 5))
for h in h_values:
    entropy_values = []
    for L in tqdm(L_values, desc=f"Calculating Entropy for h={h}"):
        periodic = False
        H = ising_sparse(L, h, 1, periodic)
        l = L // 2 
        entropy = entanglement_entropy(H, L, l, highest=True)
        entropy_values.append(entropy)
    plt.plot(list(L_values), entropy_values, marker='o', label=f'h={h}')
plt.xlabel('System size L')
plt.ylabel('Entanglement entropy S(L/2, L)')
plt.title('Entanglement Entropy S(L/2, L) for Highest Excited State vs. System Size L (Open Boundary)')
plt.legend()
plt.show()

5.3 Truncation error of schmidt decomposition for L=15

In [ ]:
import numpy as np
import scipy.sparse.linalg as sla
import scipy.sparse as sparse
import matplotlib.pyplot as plt

def schmidt_decomposition(psi, L):
    midpoint = 2**(L//2)
    psi_matrix = psi.reshape(midpoint, -1)
    U, S, Vh = np.linalg.svd(psi_matrix, full_matrices=False)
    return U, S, Vh

def compute_truncated_state(U, S, Vh, k):
    S_k = np.zeros_like(S)
    S_k[:k] = S[:k]
    return U @ np.diag(S_k) @ Vh

def compute_errors(H, psi, psi_approx):
    psi = psi.flatten() 
    psi_approx = psi_approx.flatten()  
    norm_psi = np.linalg.norm(psi)
    norm_psi_approx = np.linalg.norm(psi_approx)
    E_exact = (psi.conj().T @ (H @ psi)) / norm_psi**2 
    E_approx = (psi_approx.conj().T @ (H @ psi_approx)) / norm_psi_approx**2

    return np.linalg.norm(psi - psi_approx), np.abs(E_exact - E_approx)
L = 15 
h = 0.4
J = 1.0
H_sparse = ising_sparse(L,h,J,periodic) 
eigvals, eigvecs = sla.eigsh(H_sparse, k=1, which='SA')
psi_original = eigvecs[:, 0]
U, S, Vh = schmidt_decomposition(psi_original, L)
ks = [2**i for i in range(L//2 + 1)]
errors = []
energy_diffs = []

for k in ks:
    psi_approx = compute_truncated_state(U, S, Vh, k)
    psi_approx = psi_approx.flatten() 
    error, energy_diff = compute_errors(H_sparse, psi_original, psi_approx)
    errors.append(error)
    energy_diffs.append(energy_diff)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(ks, errors, '-o')
plt.xlabel('k')
plt.ylabel('Frobenius norm error')
plt.xscale('log')

plt.subplot(1, 2, 2)
plt.plot(errors, energy_diffs, '-o')
plt.xlabel('Frobenius norm error')
plt.ylabel('Energy difference $\Delta E$')
plt.show()


5.4 Entaglement entropy of highly excited states

In [ ]:
Dense Hamiltonian diagonilzation

In [ ]:
def ising_hamiltonian_dense(L, h, periodic):
    H = np.zeros((2**L, 2**L))
    for b in range(2**L):
        for j in range(1, L+1):
            a = b ^ (1 << (j - 1))
            H[a, b] -= h
    for a in range(2**L):
        for j in range(1, L):
            if a & (1 << j) == (a & (1 << (j - 1))) * 2:
                H[a, a] -= 1
            else:
                H[a, a] += 1
        if periodic:
            if (a & (1 << (L - 1))) == ((a & (1 << 0)) * (2**(L - 1))):
                H[a, a] -= 1
            else:
                H[a, a] += 1
    return H

Computing entaglement entropy for a highly excited state

In [ ]:
from qutip import Qobj, entropy_vn
import numpy as np
import math
import scipy.linalg as la

def entanglement_entropy_of_excited_state(H_dense, L, l):
    eigvals, eigvecs = la.eigh(H_dense)
    mid_index = len(eigvals) // 2 
    excited_state_vec = eigvecs[:, mid_index]
    excited_state = Qobj(excited_state_vec, dims=[[2] * L, [1]])
    subspace_dims = [2] * L 
    subsystem_A = list(range(l))
    rho_A = excited_state.ptrace(subsystem_A)
    entropy = entropy_vn(rho_A, base=np.e)
    return entropy
L_values =[7, 8,9,10,11] 
h = 0.4
periodic = True
for L in L_values:
    H_dense = ising_hamiltonian_dense(L, h, periodic)
    S_l_L = entanglement_entropy_of_excited_state(H_dense, L, math.ceil(L / 2))
    print(f"Entanglement entropy S({math.ceil(L / 2)};{L}) = {S_l_L}")

5.5 MPS Approximation

In [ ]:
import numpy as np
from scipy.linalg import svd

def compute_mps(wavefunction, max_bond_dim):
    L = int(np.log2(wavefunction.shape[0]))
    mps_tensors = []
    current_tensor = wavefunction
    sz = current_tensor.shape[0] * 2
    matrix = current_tensor.reshape(2, 2**(L-1))
    for i in range(L - 1):
        U, s, Vh = svd(matrix, full_matrices=False)
        min_dim = min(len(s), max_bond_dim)
        U = U[:, :min_dim]
        s = s[:min_dim]
        Vh = Vh[:min_dim, :]
        if i == 0:
            mps_tensors.append(U)
        else:
            mps_tensors.append(U.reshape((-1, 2, min_dim)))
        if i<L-2:
            matrix=(np.diag(s)@Vh).reshape(2 * min_dim,-1)
        else:
            mps_tensors.append(np.diag(s)@Vh)
    return mps_tensors
H=ising_sparse(6,1,1, False)
eigvals, eigvecs = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
wavefunction = eigvecs
mps_approx = compute_mps(wavefunction, max_bond_dim=4)
for i, tensor in enumerate(mps_approx):
    print(f"Tensor {i} shape: {tensor.shape}")

Plot of storage 

In [ ]:
H=ising_sparse(10,1.3, 1,False)
L=10
psi = scipy.sparse.linalg.eigsh(H, k=1, which='SA')[1]
K = range(1, 2**L//2)
storage = []         
for k in K:
    mps_psi = compute_mps(psi, k)
    total_storage = 0
    for tensor in mps_psi:
        total_storage += np.prod(tensor.shape) 
    storage.append(total_storage)
storage = 100*(np.array(storage))/2**L
plt.figure(figsize=(10, 5))
plt.plot(K, storage, marker='o')
plt.xlabel('Bond Dimension $k$')
plt.ylabel('Storage % of initial')
plt.title('Storage requirements versus bond dimensions $k$')
plt.grid(True)
plt.gca().invert_xaxis()
plt.show()

reconstructing the wavefunction using contraction

pushing for large Ls large and k=4

In [ ]:
H=ising_sparse(10,0.4,1,False)
psi = scipy.sparse.linalg.eigsh(H, k=1, which='SA')[1].reshape(-1)
A = np.sqrt(np.sum(np.abs(psi)**2))
psi = psi/A
k=2**2
MPS = compute_mps(psi, k)

def contract_mps_tensors(mps_tensors):
    state = mps_tensors[0]

    for i, tensor in enumerate(mps_tensors[1:], 1):
        if i == len(mps_tensors) - 1:
            state = np.einsum('...j,jk->...k', state, tensor)
        else:
            state = np.einsum('...j,jkl->...kl', state, tensor)

    return state.flatten()


for mp in MPS:
    print(mp.shape)

    final_psi = contract_mps_tensors(MPS)
A = np.sqrt(np.sum(np.abs(final_psi)**2))
final_psi = final_psi/A



print(psi)
print(final_psi)

Computing the overlap for L=10 and k=4

In [ ]:
import numpy as np
import scipy.sparse.linalg as sla
import scipy.linalg as la
import scipy.sparse as sparse

def ising_hamiltonian_dense(L, h, periodic):
    H = np.zeros((2**L, 2**L))
    for b in range(2**L):
        for j in range(1, L+1):
            a = b ^ (1 << (j - 1))
            H[a, b] -= h
    for a in range(2**L):
        for j in range(1, L):
            if a & (1 << j) == (a & (1 << (j - 1))) * 2:
                H[a, a] -= 1
            else:
                H[a, a] += 1
        if periodic:
            if (a & (1 << (L - 1))) == ((a & (1 << 0)) * (2**(L - 1))):
                H[a, a] -= 1
            else:
                H[a, a] += 1
    return H
L = 10
h = 0.4
periodic = False
H_dense = ising_hamiltonian_dense(L, h, periodic)
eigvals, eigvecs = la.eigh(H_dense)
psi_original = eigvecs[:, 0]
A = np.sqrt(np.sum(np.abs(psi_original)**2))
psi_original = psi_original / A
overlap = np.abs(np.vdot(psi_original, final_psi))
print("Overlap: ", overlap)

5.5.2 MPS Canonical form

In [ ]:
import numpy as np
from scipy.linalg import svd

def left_canonicalize(tensor, max_bond_dim):
    shape = tensor.shape
    reshaped_tensor = tensor.reshape(shape[0], -1)
    U, s, Vh = svd(reshaped_tensor, full_matrices=False)
    min_dim = min(len(s), max_bond_dim)
    U = U[:, :min_dim]
    s = s[:min_dim]
    Vh = Vh[:min_dim, :]
    return U, s, Vh

def right_canonicalize(tensor, max_bond_dim):
    shape = tensor.shape
    reshaped_tensor = tensor.reshape(-1, shape[-1])
    U, s, Vh = svd(reshaped_tensor, full_matrices=False)
    min_dim = min(len(s), max_bond_dim)
    U = U[:, :min_dim]
    s = s[:min_dim]
    Vh = Vh[:min_dim, :]
    return U, s, Vh

In [ ]:
def compute_schmidt_coefficients(tensor, direction):
    if direction == 'left':
        _, s, _ = svd(tensor.reshape(tensor.shape[0], -1), full_matrices=False)
    else:
        _, s, _ = svd(tensor.reshape(-1, tensor.shape[-1]), full_matrices=False)

    return s

def canonicalize_and_compute_schmidt(mps_tensors, max_bond_dim):
    L = len(mps_tensors)
    schmidt_coefficients = []
    for i in range(L - 1):
        U, s, Vh = left_canonicalize(mps_tensors[i], max_bond_dim)
        mps_tensors[i] = U
        schmidt_coefficients.append(s) 
        next_tensor_shape = mps_tensors[i + 1].shape
        mps_tensors[i + 1] = np.dot(np.diag(s), Vh).reshape(min(len(s), max_bond_dim), next_tensor_shape[1], -1)
    for i in reversed(range(1, L)):
        U, s, Vh = right_canonicalize(mps_tensors[i], max_bond_dim)
        mps_tensors[i] = Vh
        schmidt_coefficients.append(s)
        prev_tensor_shape = mps_tensors[i - 1].shape
        mps_tensors[i - 1] = np.dot(U, np.diag(s)).reshape(prev_tensor_shape[0], -1, min(len(s), max_bond_dim))
    orthogonality_center = mps_tensors[L // 2]

    return mps_tensors, schmidt_coefficients, orthogonality_center

In [ ]:
import numpy as np

def initialize_fixed_mps(L):
    mps_tensors = []

    # First tensor
    A1_up = np.array([[1, 1]]) / np.sqrt(2)
    A1_down = np.array([[1, -1]]) / np.sqrt(2)
    mps_tensors.append(A1_up)
    mps_tensors.append(A1_down)

    # Intermediate tensors
    for _ in range(2, L - 1):
        A_mid_up = np.array([[1, 1], [1, 0]]) / np.sqrt(2)
        A_mid_down = np.array([[0, 0], [1, -1]]) / np.sqrt(2)
        mps_tensors.append(A_mid_up)
        mps_tensors.append(A_mid_down)

    # Last tensor
    A_last_up = np.array([[1], [0]])
    A_last_down = np.array([[0], [1]])
    mps_tensors.append(A_last_up)
    mps_tensors.append(A_last_down)

    return mps_tensors

In [ ]:
H = ising_sparse(10, 1, 1, False)
eigvals, eigvecs = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
wavefunction = eigvecs
mps_tensors = compute_mps(wavefunction, max_bond_dim=2)
canonical_mps, schmidt_coefficients, orthogonality_center = canonicalize_and_compute_schmidt(mps_tensors, max_bond_dim=4)
for i, tensor in enumerate(canonical_mps):
    print(f"Tensor {i} shape: {tensor.shape}")
for i, coeffs in enumerate(schmidt_coefficients):
    print(f"Schmidt coefficients across bond {i}: {coeffs}")
print("Orthogonality center shape:", orthogonality_center.shape)